In [1]:
import pandas as pd
from datetime import date
import os

In [2]:
tDay=date.today()

In [3]:
quote='quote borrow: 1280-HK / 2598-HK / 2522-HK / 2473-HK / 2490-HK / 2495-HK / 2582-HK / 1196-HK / 7226-HK / 7552-HK / 917-HK / 601989-CH / 600418-CH'

In [4]:
text='''
CICC HT cover 103.5K shares 601988-CH limit 5.37, POV 8% passive 
CICC HT cover 10K shares 688256-CH limit 624.80, POV 3% passive 
CICC cover 1mn shares 2202-HK limit 5.63, POV 5% passive 
CLSA cover 6,964,091 shares 601328-CH limit 7.20, DESK POV 5% passive 
CLSA short 600K shares 7552-HK limit 1.906, POV 6% near-touch 
CLSA BUY 885.5K shares 291-HK limit 27.45, POV 5% near-touch
CLSA short 1mn shares 1919-HK limit 12.64, POV 5% passive 
CLSA short 1,302,500 shares 9636-HK limit 45.0, POV 10% passive 
JEFS cover 358K shares 2333-HK limit 13.96, POV 3% passive 
JEFC short 1mn shares 1113-HK limit 31.60, POV 6%
JEFC short 200K shares 2477-HK limit 22.85, POV 10%
JEFC short 39K shares 9636-HK limit 42.85, POV 8% 
JEFC short 58K shares 1196-HK limit 7.15, POV 8% 
Maybank short 1mn shares 1-HK limit 45.60, POV 5% passive 
MBSO BUY 40K shares 002594-C2 limit 368.96, POV 2% passive
'''

In [5]:
text2=text.replace("  "," ").replace(" shares","").replace(" limit","").replace(",","").replace("CLSA Desk","CLSA_HT").replace("DESK POV","DESK").replace("Desk POV","DESK").replace(" HT","_HT").replace(" (low-touch)","").replace("~","").replace('passive ','passive').replace('MACQ','MEEX').replace('NEO','POV').replace('below ','1-').replace('near touch','neartouch').replace('near-touch ','NearTouch').replace('near-touch','NearTouch').replace('Low','5-10%').replace('Mid','10-20%').replace('High','15-30%').replace('Maybank','MKEA').replace('MBSO','MBSO_HT')
tradelist=[trades.rstrip().split(" ") for trades in text2.split("\n") if trades != ""]
#tradelist

In [6]:
cimblist=[ " ".join(x[1:]) for x in tradelist if "CIMB" in x]
for order in cimblist:
    print(order)

In [7]:
header=['Broker','Side','Quantity','Ticker','Price','Strategy','Vol(%)','Style']

In [8]:
def qty_conv(q):
    if q[-2:].lower() in ['mn','mm'] :
        qty=int(float(q[:-2])*1000000)
    elif q[-1].lower()=='k':
        qty=int(float(q[:-1])*1000)
    else:
        qty=int(q)
    
    return qty
def price_conv(p):
    if str(p)[-1].lower()=='k':
        p=float(p[:-1])*1000
    return p

In [9]:
tradeframe=pd.DataFrame(tradelist,columns=header)
#tradeframe=pd.DataFrame(tradelist)
#tradeframe

In [10]:
tradeframe['Ticker']=tradeframe['Ticker'].map(lambda x: x.replace("-"," ")+" EQUITY")
tradeframe['Quantity']=tradeframe['Quantity'].map(qty_conv)
tradeframe['Price']=tradeframe['Price'].map(price_conv)
tradeframe['Vol(%)']=tradeframe['Vol(%)'].map(lambda x: x.replace("%","") if x is not None else x)
tradeframe['Side']=tradeframe['Side'].map(lambda x:x.upper())
tradeframe['Side']=tradeframe['Side'].map({'SHORT':'SHRT','COVER':'COVR',"BUY":"BUY","SELL":'SELL'})
#tradeframe

In [11]:
brokers=['CLSA','CLSA_HT','Huatai','Huatai_HT','CICC_HT','CICC','Haitong','Haitong_HT','MEEX','MKEA','JEFS','JEFC','MBSO_HT']

In [12]:
tradeframe2=tradeframe[tradeframe['Broker'].isin(brokers)]
#tradeframe2
tradeframe2['Style']=tradeframe2.apply(lambda row: row['Style'].replace('passive','neartouch') if row['Broker']=='MEEX' and row['Style']=='passive' else row['Style'], axis=1)
tradeframe2

,Broker,Side,Quantity,Ticker,Price,Strategy,Vol(%),Style
0,CICC_HT,COVR,103500,601988 CH EQUITY,5.37,POV,8,passive
1,CICC_HT,COVR,10000,688256 CH EQUITY,624.80,POV,3,passive
2,CICC,COVR,1000000,2202 HK EQUITY,5.63,POV,5,passive
3,CLSA,COVR,6964091,601328 CH EQUITY,7.20,DESK,5,passive
4,CLSA,SHRT,600000,7552 HK EQUITY,1.906,POV,6,NearTouch
5,CLSA,BUY,885500,291 HK EQUITY,27.45,POV,5,NearTouch
6,CLSA,SHRT,1000000,1919 HK EQUITY,12.64,POV,5,passive
7,CLSA,SHRT,1302500,9636 HK EQUITY,45.0,POV,10,passive
8,JEFS,COVR,358000,2333 HK EQUITY,13.96,POV,3,passive
9,JEFC,SHRT,1000000,1113 HK EQUITY,31.60,POV,6,None


In [13]:
tradeframe2.to_csv('trades_{}.csv'.format(tDay.strftime("%Y%m%d")),index=False)

In [14]:
quoteList=quote.replace("Quote borrow: ","").replace("quote borrow: ","").replace(":","").replace("Haitong","HT").replace("Guangfa","GF").split("/ ")
#quoteList

In [15]:
def quote(broker):    
    List=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if broker in x]
    String=broker+":\nHi team, could you quote borrow on "
    if broker=='Citi'or broker=='Tiger':
        List=[x for x in List if 'HK' in x]
    for x in List:
        String=String+x.strip()+", "
    String=String+"tks\n"
    String
    return String

In [16]:
brokerList=['GF','HT','CLSA','CICC','Huatai','MACQ','MB','JEFF','Tiger','Citi']#'CICC',
for broker in brokerList:
    print(quote(broker))

GF:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

HT:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

CLSA:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

CICC:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

Huatai:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

MACQ:
Hi team, could you quote borrow on 1280-HK, 2598-HK, 2522-HK, 2473-HK, 2490-HK, 2495-HK, 2582-HK, 1196-HK, 7226-HK, 7552-HK, 917-HK, 601989-CH, 600418-CH, tks

MB:
Hi

In [17]:
cimbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "CIMB" in x]
shares=[1000000]*len(cimbList)
df=pd.DataFrame(shares,cimbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

,qty
1280-HK,1000000
2598-HK,1000000
2522-HK,1000000
2473-HK,1000000
2490-HK,1000000
2495-HK,1000000
2582-HK,1000000
1196-HK,1000000
7226-HK,1000000
7552-HK,1000000


In [18]:
mbList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if "MB" in x]
shares=[1000000]*len(mbList)
df=pd.DataFrame(shares,mbList,columns=['qty'])
#csv_path='borrow.csv'
#df.to_csv('borrow.csv')
#os.system(f'start excel "{csv_path}"') 
df

,qty
1280-HK,1000000
2598-HK,1000000
2522-HK,1000000
2473-HK,1000000
2490-HK,1000000
2495-HK,1000000
2582-HK,1000000
1196-HK,1000000
7226-HK,1000000
7552-HK,1000000


In [19]:
macqList=[x for x in quoteList if "(" not in x]+[x.split()[0] for x in quoteList if 'MACQ' in x]
shares=[1000000]*len(macqList)
macq=pd.DataFrame(shares,macqList,columns=['quantity'])
macq=macq.reset_index()
macq.columns=['ticker','quantity']
def symbol(ticker):
    if ticker[-2:]=="CH":
        if ticker[0]=="6":
            ticker=ticker.replace("CH","SH")
        else:
            ticker=ticker.replace("CH","ZK")
    if ticker[-2:]=="HK":
        ticker=ticker.zfill(7)
    ticker=ticker.replace("-",".")
    return ticker
macq['ticker']=macq['ticker'].map(lambda x: x.replace(" ",""))
macq['ticker']=macq['ticker'].map(lambda x: symbol(x))
macq.loc[macq.shape[0]]=["1211.HK",500000]
macq.loc[macq.shape[0]]=["0300.HK",500000]
macq.loc[macq.shape[0]]=["0921.HK",500000]
csv_path='macqborrow.csv'
macq.to_csv('macqborrow.csv',index=False)
#os.system(f'start excel "{csv_path}"') 
macq

,ticker,quantity
0,1280.HK,1000000
1,2598.HK,1000000
2,2522.HK,1000000
3,2473.HK,1000000
4,2490.HK,1000000
5,2495.HK,1000000
6,2582.HK,1000000
7,1196.HK,1000000
8,7226.HK,1000000
9,7552.HK,1000000
